In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import flg_yolo

fast_mode = False
if fls.is_submission:
    fast_mode = False

In [2]:
all_data = fls.load_all_train_data()
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=100
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    train_data = train_data[1:30]
    test_data = test_data[2:4]
len(train_data), len(test_data)
fls.dill_save(fls.temp_dir + 'repr.pickle', ([d.name for d in train_data], [d.name for d in test_data]))

In [3]:
importlib.reload(flg_yolo)
model = flg_yolo.YOLOModel()
model.seed = 42
model.n_epochs = 5
model.run_in_parallel = True
if fast_mode: model.n_epochs = 1
model.train(train_data, test_data)
if not fls.env=='kaggle':
    fls.dill_save(fls.model_dir+ "yolo_repr.pickle", model)

Will process approximately 3123 slices for training


Processing training motors:   0%|          | 0/347 [00:00<?, ?it/s]

Will process approximately 900 slices for validation


Processing validation motors:   0%|          | 0/100 [00:00<?, ?it/s]


Processing Summary:
- Train set: 258 tomograms, 347 motors, 3118 slices
- Validation set: 100 tomograms, 100 motors, 900 slices
- Total: 358 tomograms, 447 motors, 4018 slices

Preprocessing Complete:
- Training data: 258 tomograms, 347 motors, 3118 slices
- Validation data: 100 tomograms, 100 motors, 900 slices
- Dataset directory: /flagellar/temp//yolo_dataset/
- YAML configuration: /flagellar/temp//yolo_dataset/dataset.yaml

Ready for YOLO training!
Starting YOLO training process...
Directory status:
- Train images exists: True
- Val images exists: True
- Train labels exists: True
- Val labels exists: True
Found original dataset.yaml at /flagellar/temp//yolo_dataset/dataset.yaml
Fixing YAML paths in /flagellar/temp//yolo_dataset/dataset.yaml
Created fixed YAML at /flagellar/temp/fixed_dataset.yaml with path: /flagellar/temp//yolo_dataset/
Using YAML file: /flagellar/temp/fixed_dataset.yaml
YAML contents:
names:
  0: motor
path: /flagellar/temp//yolo_dataset/
train: images/train
val

...

        5/5      8.26G      1.871      1.331     0.9139         26        640: 100%|██████████| 195/195 [00:24<00:00,  7.99it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:03<00:00,  7.93it/s]

                   all        900        900      0.823      0.778      0.818      0.244



5 epochs completed in 0.042 hours.
Optimizer stripped from /flagellar/temp/yolo_weights/motor_detector/weights/last.pt, 52.0MB
Optimizer stripped from /flagellar/temp/yolo_weights/motor_detector/weights/best.pt, 52.0MB

Validating /flagellar/temp/yolo_weights/motor_detector/weights/best.pt...
Ultralytics 8.3.98 🚀 Python-3.11.10 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4090, 23995MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 29/29 [00:06<00:00,  4.35it/s]


                   all        900        900      0.679      0.757      0.772      0.279
Speed: 0.1ms preprocess, 4.3ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to /flagellar/temp/yolo_weights/motor_detector

Training complete!


In [4]:
# if not fls.is_submission:
#     importlib.reload(flg_yolo)    
#     inferred_test_data = model.infer(test_data[1:10])
#     print(fls.score_competition_metric(inferred_test_data, test_data))

In [5]:
inferred_test_data2 = model.infer(fls.load_all_test_data())
fls.write_submission_file(inferred_test_data2)

CUDA_VISIBLE_DEVICES= 0
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
       tomo_id  Motor axis 0  Motor axis 1  Motor axis 2
0  tomo_003acc            -1            -1            -1
1  tomo_00e047           174           547           602
2  tomo_01a877           143           639           285


In [6]:
inferred_test_data2

[Data(is_train=False, name='tomo_003acc', labels=Empty DataFrame
 Columns: []
 Index: [], labels_unfiltered=Empty DataFrame
 Columns: []
 Index: [], loaded_state='unloaded', data=None, data_shape=(0, 0, 0), voxel_spacing=nan, mean_per_slice=array([], dtype=float64), std_per_slice=array([], dtype=float64), percentiles_per_slice=array([], shape=(101, 0), dtype=float64), resize_factor=1.0, target_size=None),
 Data(is_train=False, name='tomo_00e047', labels=     z    y    x
 0  174  547  602, labels_unfiltered=Empty DataFrame
 Columns: []
 Index: [], loaded_state='unloaded', data=None, data_shape=(0, 0, 0), voxel_spacing=nan, mean_per_slice=array([], dtype=float64), std_per_slice=array([], dtype=float64), percentiles_per_slice=array([], shape=(101, 0), dtype=float64), resize_factor=1.0, target_size=None),
 Data(is_train=False, name='tomo_01a877', labels=     z    y    x
 0  143  639  285, labels_unfiltered=Empty DataFrame
 Columns: []
 Index: [], loaded_state='unloaded', data=None, data_sh